In [1]:
import os
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

In [2]:
class EmailState(TypedDict):
    # The email being processed
    email: Dict[str, Any]  # Contains subject, sender, body, etc.

    # Category of the email (inquiry, complaint, etc.)
    email_category: Optional[str]

    # Reason why the email was marked as spam
    spam_reason: Optional[str]

    # Analysis and decisions
    is_spam: Optional[bool]
    
    # Response generation
    email_draft: Optional[str]
    
    # Processing metadata
    messages: List[Dict[str, Any]]  # Track conversation with LLM for analysis

In [70]:
# Initialize our LLM with Groq
# You need to set your GROQ_API_KEY environment variable
# export GROQ_API_KEY="your-groq-api-key-here"

# Initialize the Groq model
model = ChatGroq(
    model="llama-3.1-8b-instant",  # You can change this to other Groq models
    temperature=0
)

def read_email(state: EmailState):
    """Alfred reads and logs the incoming email"""
    email = state["email"]
    
    # Here we might do some initial preprocessing
    print(f"Alfred is processing an email from {email['sender']} with subject: {email['subject']}")
    
    # No state changes needed here
    return {}

def _parse_json_payload(text: str) -> dict:
    # 兼容 ```json ... ``` 或 ``` ... ```；抓第一個花括號區塊
    m = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", text, flags=re.S)
    payload = m.group(1) if m else text.strip()
    return json.loads(payload)

def _normalize_email_state(state: Dict[str, Any]) -> None:
    """一致性保護：垃圾信必有 reason，合法信 reason 必為 None。"""
    if state.get("is_spam"):
        if not state.get("spam_reason") or str(state["spam_reason"]).strip() == "":
            state["spam_reason"] = "Flagged by classifier (missing reason)"
        state["email_category"] = None
    else:
        state["spam_reason"] = None
        # 合法信 category 可留白或限定白名單
        if state.get("email_category") not in ["inquiry","complaint","thank you","request","information", None]:
            state["email_category"] = None

def classify_email(state: EmailState):
    """Alfred uses an LLM to determine if the email is spam or legitimate"""
    email = state["email"]
    
    # Prepare our prompt for the LLM
    prompt = f"""
    As Alfred the butler, analyze this email and determine if it is spam or legitimate.
    
    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}
    
    You are Alfred the butler. Analyze the email and return ONLY a compact JSON object with keys:
    - is_spam: boolean
    - spam_reason: string or null
    - category: one of ["inquiry","complaint","thank you","request","information",null]

    Rules:
    - If legitimate, set is_spam=false and category accordingly; spam_reason must be null.
    - If spam, set is_spam=true and provide a concise spam_reason; category must be null.
    - Do not include any extra text.
    """
    
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    raw = response.content.strip()

    response_text = response.content.lower()
    is_spam = "spam" in response_text and "not spam" not in response_text
    
    spam_reason = None
    if is_spam and "reason:" in response_text:
        spam_reason = response_text.split("reason:")[1].strip()

    # 兼容：有些模型會用 code fence，把它剝掉
    if raw.startswith("```"):
        raw = raw.strip("`")
        # 可能是 "json\n{...}"
        if raw.startswith("json"):
            raw = raw[4:].strip()

    data = {}
    try:
        data = json.loads(raw)
        print(f"Data = {data}")
    except Exception:
        # 後備：退化為舊的關鍵字法，但盡量降低誤判
        lower = response.content.lower()
        is_spam = ("spam" in lower and "not spam" not in lower and "legitimate" not in lower and "likely legitimate" not in lower)
        category = None
        if not is_spam:
            for c in ["inquiry","complaint","thank you","request","information"]:
                if c in lower:
                    category = c
                    break
        data = {
            "is_spam": bool(is_spam),
            "spam_reason": "Heuristic fallback (no JSON from model)" if is_spam else None,
            "category": category if not is_spam else None,
        }

    # 一致性保護：避免 None/矛盾狀態
    is_spam = bool(data.get("is_spam", False))
    spam_reason = data.get("spam_reason")
    category = data.get("category")

    if is_spam:
        # 垃圾信時必須有原因
        if not spam_reason or str(spam_reason).strip() == "":
            spam_reason = "Flagged by policy/rules (missing reason from model)"
        category = None
    else:
        spam_reason = None
        if category not in ["inquiry","complaint","thank you","request","information", None]:
            category = None  # 清理不在白名單的類別

    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]

    return {
        "is_spam": is_spam,
        "spam_reason": spam_reason,
        "email_category": category,
        "messages": new_messages
    }

def handle_spam(state: EmailState):
    """Alfred discards spam email with a note"""
    print(f"Alfred has marked the email as spam. Reason: {state['spam_reason']}")
    print("The email has been moved to the spam folder.")
    
    # We're done processing this email
    return {}

def draft_response(state: EmailState):
    """Alfred drafts a preliminary response for legitimate emails"""
    email = state["email"]
    category = state["email_category"] or "general"
    
    # Prepare our prompt for the LLM
    prompt = f"""
    As Alfred the butler, draft a polite preliminary response to this email.
    
    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}
    
    This email has been categorized as: {category}
    
    Draft a brief, professional response that Mr. Hugg can review and personalize before sending.
    """
    
    # Call the LLM
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)
    
    # Update messages for tracking
    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]
    
    # Return state updates
    return {
        "email_draft": response.content,
        "messages": new_messages
    }

def notify_mr_hugg(state: EmailState):
    """Alfred notifies Mr. Hugg about the email and presents the draft response"""
    email = state["email"]
    
    print("\n" + "="*50)
    print(f"Sir, you've received an email from {email['sender']}.")
    print(f"Subject: {email['subject']}")
    print(f"Category: {state['email_category']}")
    print("\nI've prepared a draft response for your review:")
    print("-"*50)
    print(state["email_draft"])
    print("="*50 + "\n")
    
    # We're done processing this email
    return {}

In [71]:
def route_email(state: EmailState) -> str:
    """Determine the next step based on spam classification"""
    if state["is_spam"]:
        return "spam"
    else:
        return "legitimate"

In [72]:
# Create the graph
email_graph = StateGraph(EmailState)

# Add nodes
email_graph.add_node("read_email", read_email)
email_graph.add_node("classify_email", classify_email)
email_graph.add_node("handle_spam", handle_spam)
email_graph.add_node("draft_response", draft_response)
email_graph.add_node("notify_mr_hugg", notify_mr_hugg)

# Start the edges
email_graph.add_edge(START, "read_email")
# Add edges - defining the flow
email_graph.add_edge("read_email", "classify_email")

# Add conditional branching from classify_email
email_graph.add_conditional_edges(
    "classify_email",
    route_email,
    {
        "spam": "handle_spam",
        "legitimate": "draft_response"
    }
)

# Add the final edges
email_graph.add_edge("handle_spam", END)
email_graph.add_edge("draft_response", "notify_mr_hugg")
email_graph.add_edge("notify_mr_hugg", END)

# Compile the graph
compiled_graph = email_graph.compile()

In [73]:
# Example legitimate email
legitimate_email = {
    "sender": "john.smith@example.com",
    "subject": "Question about your services",
    "body": "Dear Mr. Hugg, I was referred to you by a colleague and I'm interested in learning more about your consulting services. Could we schedule a call next week? Best regards, John Smith"
}

# Example spam email
spam_email = {
    "sender": "winner@lottery-intl.com",
    "subject": "YOU HAVE WON $5,000,000!!!",
    "body": "CONGRATULATIONS! You have been selected as the winner of our international lottery! To claim your $5,000,000 prize, please send us your bank details and a processing fee of $100."
}

# Process the legitimate email
print("\nProcessing legitimate email...")
legitimate_result = compiled_graph.invoke({
    "email": legitimate_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
})

# Process the spam email
print("\nProcessing spam email...")
spam_result = compiled_graph.invoke({
    "email": spam_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
})


Processing legitimate email...
Alfred is processing an email from john.smith@example.com with subject: Question about your services

Sir, you've received an email from john.smith@example.com.
Subject: Question about your services
Category: inquiry

I've prepared a draft response for your review:
--------------------------------------------------
Here's a draft of a polite preliminary response:

"Dear Mr. Smith,

Thank you for reaching out to me and for the referral from your colleague. I'm pleased to hear that you're interested in learning more about my consulting services. I'd be more than happy to schedule a call to discuss how I can assist you.

Before we proceed, I'd like to confirm a few details. Could you please let me know what specific areas of consulting you're interested in learning more about? This will help me tailor our conversation to your needs.

Once I have this information, I'll be in touch to schedule a call at your convenience.

Best regards,
Alfred (Mr. Hugg's Butl

In [14]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key="sk-lf-d1d8310a-4266-4522-8ecc-b923320113a4",
  public_key="pk-lf-7085ff39-2153-4783-acac-731612dc20b8",
  host="https://us.cloud.langfuse.com"
)

In [54]:
from langfuse.langchain import CallbackHandler

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler()

# Process legitimate email
legitimate_result = compiled_graph.invoke(
    input={"email": spam_email, "is_spam": None, "spam_reason": None, "email_category": None, "draft_response": None, "messages": []},
    config={"callbacks": [langfuse_handler]}
)

Alfred is processing an email from winner@lottery-intl.com with subject: YOU HAVE WON $5,000,000!!!

Sir, you've received an email from winner@lottery-intl.com.
Subject: YOU HAVE WON $5,000,000!!!
Category: None

I've prepared a draft response for your review:
--------------------------------------------------
Here's a polite preliminary response to the email:

Subject: Re: YOU HAVE WON $5,000,000!!!

Dear Winner@lottery-intl.com,

Thank you for your email informing Mr. Hugg of his alleged win in the international lottery. I must express some caution and request further clarification regarding this matter. As Mr. Hugg's butler, I am responsible for ensuring the security and integrity of his personal and financial information.

Before proceeding, I would appreciate it if you could provide more information about your organization, such as your official website, contact details, and any relevant licenses or certifications. Additionally, I would like to know more about the processing fee a